## Projeto SQL

__Conectando-se ao Banco de Dados__

In [ ]:
# import libraries
import pandas as pd
from sqlalchemy import create_engine
db_config = {
 'user': 'practicum_student', # username
 'pwd': 'QnmDH8Sc2TQLvy2G3Vvh7', # password
 'host': 'yp-trainers-practicum.cluster-czs0gxyx2d8w.us-east-1.rds.amazonaws.com',
 'port': 5432, # connection port
 'db': 'data-analyst-final-project-db' # the name of the database
 }
connection_string = 'postgresql://{}:{}@{}:{}/{}'.format(db_config['user'],
                                                         db_config ['pwd'],
                                                         db_config['host'],
                                                         db_config['port'],
                                                         db_config['db'])
# Conectando-se ao Banco de Dados 2
engine = create_engine(connection_string, connect_args={'sslmode':'require'})

__Tarefa 1: Número de livros lançados depois de 1 de janeiro de 2000__

In [ ]:
# Tarefa 1
query1 = """
SELECT COUNT(*) as total_livros_pos_2000
FROM books
WHERE publication_date > '2000-01-01';
"""

result1 = pd.io.sql.read_sql(query1, con=engine)
print("Tarefa 1 - Livros lançados após 1/1/2000:")
print(result1)

Tarefa 1 - Livros lançados após 1/1/2000:
   total_livros_pos_2000
0                    819


__Tarefa 2: Número de avaliações e classificação média por livro__

In [ ]:
# Tarefa 2
query2 = """
SELECT 
    b.book_id,
    b.title,
    COUNT(r.rating_id) as numero_avaliacoes,
    ROUND(AVG(r.rating), 2) as media_classificacao
FROM books b
LEFT JOIN ratings r ON b.book_id = r.book_id
GROUP BY b.book_id, b.title
ORDER BY numero_avaliacoes DESC;
"""

result2 = pd.io.sql.read_sql(query2, con=engine)
print("\nTarefa 2 - Avaliações e classificação média por livro:")
print(result2.head(10))


Tarefa 2 - Avaliações e classificação média por livro:
   book_id                                              title  \
0      948                            Twilight (Twilight  #1)   
1      750                The Hobbit  or There and Back Again   
2      673                             The Catcher in the Rye   
3       75               Angels & Demons (Robert Langdon  #1)   
4      302  Harry Potter and the Prisoner of Azkaban (Harr...   
5      299  Harry Potter and the Chamber of Secrets (Harry...   
6      301  Harry Potter and the Order of the Phoenix (Har...   
7      722  The Fellowship of the Ring (The Lord of the Ri...   
8       79                                        Animal Farm   
9      300  Harry Potter and the Half-Blood Prince (Harry ...   

   numero_avaliacoes  media_classificacao  
0                160                 3.66  
1                 88                 4.13  
2                 86                 3.83  
3                 84                 3.68  
4       

__Tarefa 3: Editora com mais livros com mais de 50 páginas__

In [ ]:
# Tarefa 3
query3 = """
SELECT 
    p.publisher,
    COUNT(b.book_id) as total_livros
FROM books b
JOIN publishers p ON b.publisher_id = p.publisher_id
WHERE b.num_pages > 50
GROUP BY p.publisher
ORDER BY total_livros DESC
LIMIT 1;
"""

result3 = pd.io.sql.read_sql(query3, con=engine)
print("\nTarefa 3 - Editora com mais livros >50 páginas:")
print(result3)


Tarefa 3 - Editora com mais livros >50 páginas:
       publisher  total_livros
0  Penguin Books            42


__Tarefa 4: Autor com maior média de classificação (mínimo 50 avaliações)__

In [ ]:
# Tarefa 4
query4 = """
WITH autores_com_mais_50_avaliacoes AS (
    SELECT 
        a.author_id,
        a.author,
        COUNT(rat.rating_id) as total_avaliacoes,
        ROUND(AVG(rat.rating), 2) as media_classificacao
    FROM authors a
    JOIN books b ON a.author_id = b.author_id
    JOIN ratings rat ON b.book_id = rat.book_id
    GROUP BY a.author_id, a.author
    HAVING COUNT(rat.rating_id) > 50
)
SELECT 
    author,
    media_classificacao,
    total_avaliacoes
FROM autores_com_mais_50_avaliacoes
ORDER BY media_classificacao DESC
LIMIT 1;
"""

result4 = pd.io.sql.read_sql(query4, con=engine)
print("\nTarefa 4 - Autor com maior média de classificação:")
print(result4)


Tarefa 4 - Autor com maior média de classificação:
                       author  media_classificacao  total_avaliacoes
0  J.K. Rowling/Mary GrandPré                 4.29               312


__Tarefa 5: Número médio de reviews entre usuários que avaliaram mais de 50 livros__

In [ ]:
# Tarefa 5
query5 = """
WITH usuarios_ativos AS (
    -- Usuários que classificaram mais de 50 livros
    SELECT 
        username
    FROM ratings
    GROUP BY username
    HAVING COUNT(DISTINCT book_id) > 50
),
reviews_por_usuario AS (
    -- Contar reviews desses usuários ativos
    SELECT 
        ua.username,
        COUNT(rev.review_id) as total_reviews
    FROM usuarios_ativos ua
    LEFT JOIN reviews rev ON ua.username = rev.username
    GROUP BY ua.username
)
SELECT 
    ROUND(AVG(total_reviews), 2) as media_reviews_por_usuario_ativo
FROM reviews_por_usuario;
"""

result5 = pd.io.sql.read_sql(query5, con=engine)
print("\nTarefa 5 - Média de reviews entre usuários ativos:")
print(result5)


Tarefa 5 - Média de reviews entre usuários ativos:
   media_reviews_por_usuario_ativo
0                            24.33


## Conclusão

Com base nos resultados das consultas, podemos extrair as seguintes informações:

1. Após primeiro de janeiro de 2000 foram lançados 819 títulos
2. Dos livros mais populares, apenas o mais popular obteve mais de 90 avaliações. Em relação as classaificações médias dos títulos, todos os livros mais populares ficaram com médias acima de 3.5
3. A editora com mais livros com mais de 50 páginas foi a Penguin Books, com 42 títulos lançados
4. Dentre os autores, a autora com melhor classificação média foi J.K. Rowling/Mary GrandPré, com uma classificação de 4.29 e um total de 312 avaliações
5. A média de reviews dos usuários ativos ficou em 24.33

__Insights__
- 819 títulos pós-2000 indicam um catálogo atualizado e relevante
- Classificações acima de 3.5 indicam satisfação geral, mas com espaço para crescimento
- J.K. Rowling/Mary GrandPré é a autora de maior qualidade reconhecida
- Média de 24.33 reviews por usuário ativo indica que há espaço para um engajamento maior dos usuários, já que muitos deles classificam, mas não fazem avaliações desses livros